In [5]:
import cv2

import cv2
import numpy as np
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468 * 3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, face, lh, rh])


class VideoCamera:
    def __init__(self):
        # Initialize the camera
        self.cap = cv2.VideoCapture(0)
        self.mp_holistic = mp.solutions.holistic

    def __del__(self):
        # Release the camera when the object is destroyed
        self.cap.release()

    def get_frame(self):
        ret, frame = self.cap.read()
        image, results = mediapipe_detection(frame, self.mp_holistic())
        keypoints = extract_keypoints(results)

        # Process the frame as needed

        # Encode the frame as JPEG
        _, jpeg = cv2.imencode('.jpg', image)
        return jpeg.tobytes()
